In [15]:
import pandas as pd
import time

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
import chardet

with open("../data/raw/indigenous_business_directory.csv", "rb") as file:
    raw_data = file.read()
    result = chardet.detect(raw_data)
    print(result)

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [17]:
ind_business = pd.read_csv("../data/raw/indigenous_business_directory.csv", encoding="ISO-8859-1")

In [18]:
fed_contract_2021 = pd.read_csv("../data/raw/search_contracts_2021.csv", low_memory=False)
fed_contract_2022 = pd.read_csv("../data/raw/search_contracts_2022.csv", low_memory=False)
fed_contract_2023 = pd.read_csv("../data/raw/search_contracts_2023.csv", low_memory=False)
print(len(fed_contract_2021),len(fed_contract_2022),len(fed_contract_2023))

82087 87891 81163


## Scrape more Indigenous Businesses (CCIB)

In [2]:
def init_service(headless=False):
    # Initialize
    driver_path = '/Users/zhangliyao/Downloads/chromedriver'
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--enable-automation')
    if headless:
        chrome_options.add_argument('--headless')
    
    driver = webdriver.Chrome(service=Service(executable_path=driver_path), options=chrome_options)
    driver.implicitly_wait(10)
    driver.get('https://www.ccab.com/main/ccab_member/?qcompanyname&qccabindustry&qccabprogram&qccabterritory&qccabmembershiptype&qccabindex&s')
    time.sleep(2)
    driver.fullscreen_window()
    
    return driver

In [13]:
driver = init_service()

In [14]:
num_total = 0
has_next = True
#limit = 50
company_names = []
while has_next:
    company_list = driver.find_elements(By.CLASS_NAME, 'article')
    num_total += len(company_list)
    for element in company_list:
        company_names.append(element.text)
    try:
        driver.find_element(By.CLASS_NAME, 'next.page-numbers').click()
        driver.fullscreen_window()
        time.sleep(1)
    except:
        has_next = False
        
print("companies found:", num_total)

df = pd.DataFrame(company_names, columns=['Company Name'])
df.to_excel('CCIB_members.xlsx', index=False)

companies found: 2561


In [ ]:
search_bar = driver.find_element(By.NAME, 'detailedSearch')
search_bar.send_keys("AKLAK AIR")
try:
    search_button = driver.find_element(By.NAME, 'buttonFind')
except:
    search_button = driver.find_element(By.NAME, 'action:Search')

search_button.click()
driver.fullscreen_window()

results = driver.find_elements(By.CLASS_NAME, 'list-group-item')

if len(results) > 0:
    results[0].find_element(By.TAG_NAME, 'a').click()
    driver.fullscreen_window()

## Data Cleaning

In [20]:
# Sanity check
## unique id
fed_contract_2021 = fed_contract_2021[fed_contract_2021['contract_value'] >= 10000]
fed_contract_2022 = fed_contract_2022[fed_contract_2022['contract_value'] >= 10000]
fed_contract_2023 = fed_contract_2023[fed_contract_2023['contract_value'] >= 10000]
## contract value
fed_contract_2021 = fed_contract_2021.drop_duplicates(subset='procurement_id', keep='first')
fed_contract_2022 = fed_contract_2022.drop_duplicates(subset='procurement_id', keep='first')
fed_contract_2023 = fed_contract_2023.drop_duplicates(subset='procurement_id', keep='first')
print(len(fed_contract_2021),len(fed_contract_2022),len(fed_contract_2023))

66502 72603 70325


In [21]:
fed_contract_2023['vendor_name'] = fed_contract_2023['vendor_name'].str.lower()
ind_business['Company Operating Name'] = ind_business['Company Operating Name'].str.lower()
ind_business['Company Legal Name'] = ind_business['Company Legal Name'].str.lower()

# Match vendor with indigenous business directory
df_merge1 = pd.merge(fed_contract_2023, ind_business, left_on='vendor_name', right_on='Company Operating Name')
print(len(df_merge1))

df_merge2 = pd.merge(fed_contract_2023, ind_business, left_on='vendor_name', right_on='Company Legal Name')
print(len(df_merge2))

# Concatenate the two merge results
df_combined = pd.concat([df_merge1, df_merge2])

# Drop duplicate rows (e.g., based on 'vendor_name' to avoid double counting the same vendor)
df_final = df_combined.drop_duplicates(subset='procurement_id')

# Output the length of the final merged DataFrame
print(len(df_final))

1356
1478
1710


In [154]:
df_final.contract_value.sum() / fed_contract_2023.contract_value.sum() * 100

0.9144815900713886

In [155]:
len(df_final)/len(fed_contract_2023) * 100

2.4315677212939923

In [130]:
#df_final.to_excel('/Users/zhangliyao/Downloads/matched_contracts.xlsx', index=False)

## Overall Analysis

In [23]:
fed_contract_2023.head()

,reference_number,procurement_id,vendor_name,vendor_postal_code,buyer_name,contract_date,year,economic_object_code,description_eng,contract_period_start,delivery_date,contract_value,original_value,amendment_value,comments_eng,additional_comments_eng,agreement_type_code_en,commodity_type_en,commodity_code,country_of_vendor_en,solicitation_procedure_en,limited_tendering_reason_en,trade_agreement_exceptions_en,indigenous_business_excluding_psib_en,intellectual_property_en,potential_commercial_exploitation_en,former_public_servant_en,standing_offer_number,instrument_type_en,ministers_office_en,number_of_bids,reporting_period,owner_org_en,amendment_no,procurement_count,aggregate_total,pseudo_procurement_id,contract_value_range_en,trade_agreement_en,socioeconomic_indicator_en,article_6_exceptions_en,indigenous_business_en,original_value_range_en,amendment_value_range_en,award_criteria_en,contracting_entity_en,land_claims_en,id
0,C-2022-2023-Q4-00001,3000758543,genome quebec,H3B,"Bergeron, Gilles",2023-01-19T00:00:00Z,2023,430,Scientific services,2023-01-19T00:00:00Z,2023-03-31T00:00:00Z,40137.60,40137.60,0.0,-,-,-,Service,H300,Canada,Competitive – Open Bidding,['None'],['None'],No,No IP Terms in Contract,No,No,7000003345,Contract,No,0,2022-2023-Q4,Agriculture and Agri-Food Canada,0,1.0,40137.60,3000758543,-,"['Canada-Chile Free Trade Agreement', 'Canada-...",NaN,Not applicable,1. None,-,-,Not applicable,Call-up or Contract Against a Standing Offer o...,['Not Applicable'],"aafc-aac,C-2022-2023-Q4-00001"
1,C-2022-2023-Q4-00003,3000762275,hartland agromart ltd,E7P,"Robichaud, Carrie",2023-03-14T00:00:00Z,2023,1121,"Salt, sulphur, ores, earth and stone",2023-03-14T00:00:00Z,2023-03-31T00:00:00Z,13000.00,13000.00,0.0,-,-,-,Good,6800,Canada,Competitive – Traditional,['None'],['None'],No,No IP Terms in Contract,No,No,-,Contract,No,3,2022-2023-Q4,Agriculture and Agri-Food Canada,0,1.0,13000.00,3000762275,-,['None'],NaN,Not applicable,1. None,-,-,Lowest Price,Contract awarded by the department.,['Not Applicable'],"aafc-aac,C-2022-2023-Q4-00003"
2,C-2022-2023-Q4-00005,3000759579,glacier farmmedia limited partnership,R3C,"BARBER, NICKI",2023-02-02T00:00:00Z,2023,361,Electronic subscriptions and electronic public...,2023-02-02T00:00:00Z,2023-03-31T00:00:00Z,11566.80,11566.80,0.0,-,-,-,Service,D317,Canada,Non-Competitive,['None'],['None'],No,No IP Terms in Contract,No,No,-,Contract,No,1,2022-2023-Q4,Agriculture and Agri-Food Canada,0,1.0,11566.80,3000759579,-,['None'],NaN,Does not exceed s.6 (b) Government Contracts R...,1. None,-,-,Not applicable,Contract awarded by the department.,['Not Applicable'],"aafc-aac,C-2022-2023-Q4-00005"
3,C-2022-2023-Q4-00006,3000759047,sdl international (canada) inc,H3A,"McNeil, Connor",2023-01-26T00:00:00Z,2023,583,License/Maintenance fees for application softw...,2023-01-26T00:00:00Z,2023-04-01T00:00:00Z,12954.32,12954.32,0.0,-,-,-,Service,JX70,Canada,Non-Competitive,['None'],['None'],No,No IP Terms in Contract,No,No,EN578-100808/347/EE,Contract,No,1,2022-2023-Q4,Agriculture and Agri-Food Canada,0,1.0,12954.32,3000759047,-,['None'],NaN,Does not exceed s.6 (b) Government Contracts R...,1. None,-,-,Not applicable,Call-up or Contract Against a Standing Offer o...,['Not Applicable'],"aafc-aac,C-2022-2023-Q4-00006"
4,C-2022-2023-Q4-00008,3000758553,genome quebec,H3B,"Bergeron, Gilles",2023-01-19T00:00:00Z,2023,430,Scientific services,2023-01-19T00:00:00Z,2023-03-31T00:00:00Z,26758.40,26758.40,0.0,-,-,-,Service,H300,Canada,Competitive – Open Bidding,['None'],['None'],No,No IP Terms in Contract,No,No,7000003345,Contract,No,0,2022-2023-Q4,Agriculture and Agri-Food Canada,0,1.0,26758.40,3000758553,-,"['Canada-Chile Free Trade Agreement', 'Canada-...",NaN,Not applicable,1. None,-,-,Not applicable,Call-up or Contract Against a Standing Offer o...,['Not Applicable'],"aafc-aac,C-2022-2023-Q4-00008"


In [33]:
fed_contract_2021.award_criteria_en.value_counts()

award_criteria_en
-                                                             58932
Not applicable                                                19477
Lowest Price                                                   1672
Highest Combined Rating of Technical Merit and Price            814
Variations or combinations of the above methods                 235
Lowest Cost-per-Point                                            42
Highest Technical Merit within a Stipulated Maximum Budget       33
Contract awarded by the department.                               2
Name: count, dtype: int64

In [32]:
fed_contract_2021.indigenous_business_en.value_counts()

indigenous_business_en
-                         52700
1. None                   27970
3. Voluntary Set-Aside      440
2. Mandatory Set-Aside       97
Name: count, dtype: int64

In [31]:
fed_contract_2021.indigenous_business_excluding_psib_en.value_counts()

indigenous_business_excluding_psib_en
No                                                79457
Yes                                                1738
Not Applicable (discontinued as of 2022-01-01)       11
No IP Terms in Contract                               1
Name: count, dtype: int64

In [30]:
fed_contract_2021.commodity_code.value_counts()

commodity_code
R019        2876
D302        2261
N9130E      1900
D302A       1734
7030        1529
R109        1487
U099        1186
81110000    1079
N7030        970
N6505        891
D399         865
E199         845
6600         762
K105         740
6640         738
M190A        722
N7110        688
5179FB       607
78182000     604
JX70         576
U099D        554
N8900        536
R019F        532
N6640        512
H300         460
N2310        455
D317         442
7025         442
7110         440
R010         412
D301         407
J019A        404
J019         401
5129         390
R019AB       388
B109         387
7035         386
R199         368
K105AO       365
9100         357
N9999        356
80160000     354
G001         350
K100A        345
6800         325
JX1680A      316
R019BF       314
R199B        310
V200         301
AZ11         294
5164         293
K100         287
7010         284
5179         274
5131         272
5161         265
N9140C       258
T004         243

In [11]:
fed_contract_2021.indigenous_business_en.unique()

array(['-', '1. None', '2. Mandatory Set-Aside', '3. Voluntary Set-Aside'],
      dtype=object)